In [2]:
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers import Regex
import re
from collections import Counter

In [156]:
def read_edits(path):
    with open(path) as f:
        return [x.strip() for x in f.readlines()]

In [158]:
def uniq_op(edits):
    all_single_edits = []
    for edit in edits:
        # single_edits = re.findall(r'I_\[.*?\]+|A_\[.*?\]+|R_\[.*?\]+|K\*|.', edit)
        single_edits = re.findall(r'I_\[.*?\]+|A_\[.*?\]+|R_\[.*?\]+|K\*|.', edit)
        all_single_edits.extend([e for e in single_edits if e != ' '])
    return Counter(all_single_edits)

In [159]:
edits = read_edits('qalb14_train_edits.seqs.txt')
single_edits = uniq_op(edits)

In [161]:
with open('atomic_edits.txt', 'w') as f:
    for k, v in single_edits.items():
        f.write(f'{k}\t{v}\n')

In [139]:
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = WhitespaceSplit()
trainer = WordPieceTrainer(special_tokens=single_edits)

In [140]:
train_file = ['qalb14_train_edits.seqs.txt']

In [141]:
tokenizer.train(train_file, trainer)

In [142]:
tokenizer.save("tokenizer_edits.json")

In [143]:
tokenzier = Tokenizer.from_file('tokenizer_edits.json')

In [148]:
bla = tokenizer.encode('I_[يري]KKKKKKKKKKR_[و]R_[ن]')

In [149]:
bla.tokens

['I_[يري]', 'KKKKKK', '##KK', '##KK', 'R_[و]', 'R_[ن]']

In [34]:
def get_oov(examples, tokenizer):
    oov = set()
    for example in examples:
        words = example.split()
        for word in words:
            tokenized_word = tokenizer.tokenize(word, flatten='True')
            if '[UNK]' in tokenized_word:
                oov.add(word)

    return oov

In [39]:
with open('/scratch/ba63/arabic-text-editing/data/coda/madar/train.preproc.raw.txt') as f:
    examples = [line.strip() for line in f.readlines() if line.strip()]

In [40]:
from tokenizer import Tokenizer
tokenizer = Tokenizer('/scratch/ba63/BERT_models/bert-large-arabertv02')

In [41]:
oov = get_oov(examples, tokenizer)

In [42]:
oov

{'بيچر؟'}